In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os
import time, csv

# ========================
# 1. 크롬 옵션
# ========================
options = Options()
options.add_argument("--headless")  # 필요하면 백그라운드로 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)


# ========================
# 2. 자동 크롤링할 월 리스트
# ========================
months = [f"2023-{m:02d}-00" for m in range(1, 13)]  # 2025년 1~12월 자동


# ========================
# 3. 월별 자동 반복 크롤링
# ========================
for month in months:

    print(f"\n\n==============================")
    print(f"📅 {month} 데이터 수집 시작")
    print("==============================\n")

    url = f"https://auto.danawa.com/auto/?Work=record&Tab=Model&Month={month}"
    driver.get(url)

    time.sleep(5)

    # 브랜드별 보기 버튼 클릭
    try:
        # brand_btn = driver.find_element(By.XPATH, '/html/body/div/section/div/div/div[2]/div[3]/div[1]/div[1]/ul/li[1]/button') #현대
        brand_btn = driver.find_element(By.XPATH, '/html/body/div/section/div/div/div[2]/div[3]/div[1]/div[1]/ul/li[2]/button') #기아
        brand_btn.click()
        print("✔ 브랜드별 보기 클릭 완료")
    except:
        print("⚠ 브랜드별 버튼 클릭 실패 → 페이지 구조 변했을 수 있음")

    # 스크롤 내리기 (렌더링 도움)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(1)

    # ========================
    # 4. 테이블 로딩 대기 (최대 20초)
    # ========================
    timeout = 5
    rows = []

    for i in range(timeout):
        rows = driver.find_elements(By.CSS_SELECTOR, "table.recordTable.model tbody tr")
        if rows:
            print(f"✔ {i+1}초 후 데이터 로드 완료 ({len(rows)}개 행)")
            break
        time.sleep(1)
    else:
        print("❌ Timeout: 테이블 로드 실패")
        continue

    # ========================
    # 5. 데이터 추출 (컬럼 확장)
    # ========================

    data = []

    for row in rows:
        cols = [td.text.strip() for td in row.find_elements(By.CSS_SELECTOR, "td")]

    # 정상 행은 항상 8개 column
        if len(cols) != 8:
            continue

        순위 = cols[1].strip()
        모델명 = cols[3].strip()
        판매량 = cols[4].strip()
        점유율 = cols[5].strip()

    # 전월대비 = cols[6]
        전월_raw = cols[6].split("\n")
        전월대비 = " ".join([x.strip() for x in 전월_raw])  # ex) "7,323 419▼"

    # 전년대비 = cols[7]
        전년_raw = cols[7].split("\n")
        전년대비 = " ".join([x.strip() for x in 전년_raw])  # ex) "4,114 2,790▲"

        data.append([
            순위, "", 모델명, 판매량, 점유율,
            전월대비, 전년대비
        ])


    # ========================
    # 6. CSV 저장
    # ========================
    folder = "kia/"
    os.makedirs(folder, exist_ok=True)

    filename = os.path.join(folder, f"kia_model_sales_{month.replace('-', '_')}.csv")
    with open(filename, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow([
    "순위", "", "모델명", "판매량", "점유율",
    "전월대비", "전년대비"
])
        writer.writerows(data)

    print(f"💾 CSV 저장 완료 → {filename}")


# ========================
# 7. 브라우저 종료
# ========================
driver.quit()
print("\n🎉 모든 월 데이터 자동 크롤링 완료!")




📅 2023-01-00 데이터 수집 시작

✔ 브랜드별 보기 클릭 완료
✔ 1초 후 데이터 로드 완료 (69개 행)
💾 CSV 저장 완료 → kia/kia_model_sales_2023_01_00.csv


📅 2023-02-00 데이터 수집 시작

✔ 브랜드별 보기 클릭 완료
✔ 1초 후 데이터 로드 완료 (75개 행)
💾 CSV 저장 완료 → kia/kia_model_sales_2023_02_00.csv


📅 2023-03-00 데이터 수집 시작

✔ 브랜드별 보기 클릭 완료
✔ 1초 후 데이터 로드 완료 (75개 행)
💾 CSV 저장 완료 → kia/kia_model_sales_2023_03_00.csv


📅 2023-04-00 데이터 수집 시작



KeyboardInterrupt: 